<a href="https://colab.research.google.com/github/kazcfz/LlamaIndex-RAG/blob/main/LLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install llama-index pypdf accelerate

In [2]:
import os
import time

from llama_index import Prompt, StorageContext, load_index_from_storage, ServiceContext, VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms import LangChainLLM, HuggingFaceLLM
from llama_index.embeddings import HuggingFaceEmbedding

# from langchain.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings
# from langchain.llms import LlamaCpp

In [3]:
# read files from data_dir
pdf_path = '/content/rag_data.pdf'

filename_fn = lambda filename: {'file_name': os.path.basename(pdf_path)}
loader = SimpleDirectoryReader(input_files=[pdf_path], file_metadata=filename_fn)
documents = loader.load_data()

In [4]:
# Indexing
start_time = time.time()

embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-base-en") #Alt: thenlper/gte-base
service_context_embed = ServiceContext.from_defaults(llm=None, embed_model=embed_model, chunk_size=512)
index = VectorStoreIndex.from_documents(documents, service_context=service_context_embed)
# index.storage_context.persist(persist_dir=index_dir)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed indexing time: {elapsed_time:.2f} s")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['encoder.layer.1.intermediate.dense.bias', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.4.intermediate.dense.bias', 'encoder.layer.8.output.dense.weight', 'encoder.layer.7.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encod

LLM is explicitly disabled. Using MockLLM.
Elapsed indexing time: 11.88 s


In [5]:
# Model loading

start_time = time.time()
# llm = LangChainLLM(LlamaCpp(model_path=local_path, callback_manager=callback_manager, verbose=True, n_ctx=4096, input={"temperature": 0.7, "repeat_penalty": (1/0.85), "top_k": 40, "top_p": 0.1})) # quantized bs
llm = HuggingFaceLLM(model_name="NousResearch/Llama-2-7b-chat-hf")
text_qa_template = Prompt("""<s>[INST] <<SYS>>\nAnswer the query while referring to this provided context information: {context_str}\n<<SYS>> \n{query_str} [/INST] """)

# storage_context = StorageContext.from_defaults(persist_dir=f"./streamlit_projects/{project_name}/data")
# index = load_index_from_storage(storage_context, service_context=ServiceContext.from_defaults(llm=None, embed_model=embed_model, chunk_size=512))
service_context_llm = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=512)
index = index.as_query_engine(text_qa_template=text_qa_template, streaming=True, service_context=service_context_llm)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed loading time: {elapsed_time:.2f} s")

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Elapsed loading time: 157.10 s


In [ ]:
# Inferencing

while (True):
  # print('Querying...')
  user_query = input("User: ")
  response = index.query(user_query)
  print(response)



User: How do consultations on hypertension usually goes?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Exception in thread Thread-12 (generate):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 1673, in generate
    return self.greedy_search(
  